In [ ]:
# TensorFlow 1 버전 사용
%tensorflow_version 1.x

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

In [ ]:
# 디렉토리 변경
os.chdir('drive/My Drive/Colab Notebooks/DACON/AIFrenz_Season2/')

# 데이터 압축 풀기

In [ ]:
import zipfile

In [ ]:
# train 데이터 압축 해제
train_path = './Data/train'
if not os.path.exists(train_path):
    train_zip = zipfile.ZipFile('./Data/train.zip')
    train_zip.extractall(train_path)
    train_zip.close()

In [ ]:
# test 데이터 압축 해제
test_path = './Data/test'
if not os.path.exists(test_path):
    test_zip = zipfile.ZipFile('./Data/test.zip')
    test_zip.extractall(test_path)
    test_zip.close()

# 데이터 나누기

In [ ]:
from tqdm import tqdm

In [ ]:
import numpy as np

In [ ]:
# train 데이터 분리
if not os.path.exists('./Data/dacon_npy/train_images.npy') or not os.path.exists('./Data/dacon_npy/train_data.npy') or not os.path.exists('./Data/dacon_npy/train_labels.npy'):
    train_images = []
    train_data = []
    train_labels = []
    for fname in tqdm(os.listdir(train_path)):
        data = np.load(train_path + '/' + fname)

        image = data[:, :, :9]
        train_images.append(image.reshape(40, 40, 9).astype('float32'))

        d = data[:, :, 9:-1]
        train_data.append(d.reshape(40, 40, -1).astype('float32'))

        label = data[:, :, -1]
        label = np.where(label < 0, 0, label)
        train_labels.append(label.reshape(40, 40, 1).astype('float32'))
    np.save('./Data/dacon_npy/train_images', train_images)
    np.save('./Data/dacon_npy/train_data', train_data)
    np.save('./Data/dacon_npy/train_labels', train_labels)

In [ ]:
# test 데이터 분리
if not os.path.exists('./Data/dacon_npy/test_images.npy') or not os.path.exists('./Data/dacon_npy/test_data.npy'):
    test_images = []
    test_data = []
    for fname in tqdm(os.listdir(test_path)):
        data = np.load(test_path + '/' + fname)

        image = data[:, :, 9]
        test_images.append(image.reshape(40, 40, 9).astype('float32'))

        d = data[:, :, 9:]
        test_data.append(d.reshape(40, 40, -1).astype('float32'))
    np.save('./Data/dacon_npy/test_images', test_image)
    np.save('./Data/dacon_npy/test_data', test_data)

# 데이터 읽어오기

In [ ]:
# 저장된 train 데이터 읽어오기
train_images = np.load('./Data/dacon_npy/train_images.npy')
train_data = np.load('./Data/dacon_npy/train_data.npy')
train_labels = np.load('./Data/dacon_npy/train_labels.npy')

In [ ]:
# 저장된 test 데이터 읽어오기
test_images = np.load('./Data/dacon_npy/test_images.npy')
test_data = np.load('./Data/dacon_npy/test_data.npy')

# Keras

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

In [ ]:
model = Sequential()

# layer 1
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(40, 40, 9)))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.25))

# layer 2
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.25))

# layer 3
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.25))

# Fully Connected
model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1, activation='relu'))

In [ ]:
from sklearn.metrics import f1_score

def mae(y_true, y_pred) :
    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    over_threshold = y_true >= 0.1
    
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    remove_NAs = y_true >= 0
    
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    
    return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

def maeOverFscore_keras(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

In [ ]:
model.compile("adam", "mae", [maeOverFscore_keras])

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
callbacks_list =[EarlyStopping(patience=10), ReduceLROnPlateau('mae', 0.5, 2, 1, min_lr=0.00001)]

In [ ]:
model.fit(train_images, train_labels, epochs=2, verbose=1, callbacks=callbacks_list)